In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/nishantkushwaha-1999/llm_fine_tuning_optimization.git


Cloning into 'llm_fine_tuning_optimization'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 89 (delta 37), reused 60 (delta 18), pack-reused 0
Receiving objects: 100% (89/89), 6.36 MiB | 15.15 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [ ]:
!pip install -r ./llm_fine_tuning_optimization/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 41.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
 

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from llm_fine_tuning_optimization.Tunner import fine_tunner

In [ ]:
df = pd.read_csv("/content/final_data.csv")[:100]
df.head()

,Unnamed: 0,Disease,user_input_ques,user_input_ans
0,0,Malaria,"Hey there, for the past few days chills, vomi...",Analysing the Symptoms Malaria Consult neares...
1,1,Malaria,"Hey, facing some symptoms like chills, vomiti...",This is may be Malaria Consult nearest hospit...
2,2,Malaria,"Hello, past few days chills, vomiting, high f...",It may be due to Malaria Consult nearest hosp...
3,3,Malaria,"Hi, past few days vomiting, high fever, sweat...",Analysing the Symptoms Malaria Consult neares...
4,4,Malaria,"Hi there, facing some symptoms like chills, h...",This is may be Malaria Consult nearest hospit...


In [ ]:
tuner = fine_tunner.Llama_Tuner()

In [ ]:
train_data = tuner.to_dataset(list(df['user_input_ques']), list(df['user_input_ans']))

i = 0
for _ in train_data:
  print(_)
  i += 1
  if i >= 10:
    break

In [ ]:
model_name = 'NousResearch/Llama-2-7b-chat-hf'
save_name = './drive/MyDrive/llama-2-7b-meddata-enhanced'

quant_config = {
    'load_in_4bit': True,
    'bnb_4bit_quant_type': "nf4",
    'bnb_4bit_compute_dtype': getattr(torch, "float16"),
    'bnb_4bit_use_double_quant': False
}

lora_config = {
    'lora_alpha': 16,
    'lora_dropout': 0.5,
    'r': 64,
    'bias': "none",
    'task_type': "CASUAL_LM",
    # use None if you dont wat to use target modules.
    'target_modules': [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ]
}

train_config = {
    'output_dir': "./results",
    'num_train_epochs': 3,
    'per_device_train_batch_size': 4,
    'gradient_accumulation_steps': 1,
    'optim': "paged_adamw_32bit",
    'save_steps': 25,
    'logging_steps': 25,
    'learning_rate': 2e-4,
    'weight_decay': 0.001,
    'fp16': False,
    'bf16': False,
    'max_grad_norm': 0.3,
    'max_steps': -1,
    'warmup_ratio': 0.03,
    'group_by_length': True,
    'lr_scheduler_type': "constant",
    'report_to': "tensorboard",
    'remove_unused_columns': False
}

In [ ]:

tuner.load_model(model_name=model_name, quant_config=quant_config, lora_config=lora_config, train_config=train_config)

In [ ]:

tuner.tune_and_save(train_data, save_name)

In [ ]:
prompt = "diarrhoea"
tuner.generate_base_text(prompt)

In [ ]:
test_df = pd.read_csv("/content/test.csv")[:2]
test_dataset = tuner.to_dataset(list(test_df['question']), list(test_df['answer']))

reference = []
for item in test_dataset:
  reference.append(item['text'])

response = []
base_response = []
for ques in tqdm(test_df['question']):
  base_resp = tuner.generate_base_text(ques)
  resp = tuner.generate_text(ques)
  base_response.append(base_resp.strip())
  response.append(resp.strip())

  base_df = pd.DataFrame(base_response, columns=['response'])
  base_df.to_csv(f"./drive/MyDrive/{model_name}.csv", index=False)

  tuned_df = pd.DataFrame(response, columns=['response'])
  tuned_df.to_csv(f"./drive/MyDrive/{model_name}_tuned.csv", index=False)

bleu_score = tuner.eval_model(reference, base_response, ['bleu'])
print(f"The bleu score for base model is {bleu_score}")

bleu_score = tuner.eval_model(reference, response, ['bleu'])
print(f"The bleu score for tuned model is {bleu_score}")

rouge_score = tuner.eval_model(reference, base_response, ['rouge'])
print(f"The rouge score for base model is {bleu_score}")

rouge_score = tuner.eval_model(reference, response, ['rouge'])
print(f"The rouge score for tuned model is {bleu_score}")

rouge_score = tuner.eval_model(reference, base_response, ['ter'])
print(f"The ter score for base model is {bleu_score}")

rouge_score = tuner.eval_model(reference, response, ['ter'])
print(f"The ter score for tuned model is {bleu_score}")
